In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_excel('data.xlsx', sheet_name='2013')
df = df[(df['Variable'] == 'Studienbeeinträchtigungen') | (df['Category'] == 'Anzahl Studierende') | (df['Category'] == 'Anteil Teilnehmer (%)')]


In [6]:
df['sum'] = df.sum(axis=1, numeric_only=True)
df = df.loc[~df['Category'].isin(['Anzahl Studierende', 'Anteil Teilnehmer (%)']), ['Category', 'sum']]


In [7]:
fig = px.line_polar(df, r=df['sum'], theta=df['Category'], line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(template= 'seaborn')


fig.show()
# add total number of n
# add normalisation